In [1]:
# importing all packages and setting up variables for later use. 
#Cif regex = client identifier
# path = data source
# req = excel file type
# all_files = all files used to extract data
# ws_list = list of data from each spreadsheet
# job_titles = regex for getting job titles of staff

import itertools
import glob
from openpyxl import load_workbook
from pandas import DataFrame
import pandas as pd
import os
import numpy as np
import re
import shutil

cif_regex= r'(?<!\d)((?!201[3-7])\d{4}|\d{3})(?!\d)(?!1415)(?!1516)(?!1617)(?!1990)(?!121)(?!360)'
path =r'C:\Users\iwan.williams\Documents\Working Python Files\example_cost_summaries'
req = re.compile(r'\.xls[xm]?$')
all_files = list(filter(lambda x: req.search(x), glob.iglob(path + '/*.xls*')))
ws_list=[]
job_titles=re.compile(r'[a-zA-Z]+')

'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'


C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\reader\worksheet.py:310: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
Cannot set a frame with no defined index and a value that cannot be converted to a Series
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Cla

'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
index 0 is out of bounds for axis 0 with size 0
'Grand Total of Claimable Costs'
index 0 is out of bounds for axis 0 with size 0
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
Cannot set a frame with no defined index and a value that cannot be converted t

In [ ]:
#function to extract all spreadsheets from our server to a folder in documents. 
# If it isn't extracted, print result.

def get_cost_summaries():
    for folderName, subfolders, filenames in os.walk(r'T:\R&D\Reports'):    
        for file in filenames:
            if 'Costs Summary' in file or 'Cost Summary' in file:
                try: shutil.copy(os.path.join(folderName, file), 
                    r'C:\Users\iwan.williams\Documents\Working Python Files\example_cost_summaries')
                except: 
                    print(folderName, file)

In [ ]:
# Functions to extract data from spreadsheets and remove empty spaces
def get_data(ws):
        for row in ws.values:
            row_it=iter(row)
            for cell in row_it:
                if cell is not None:
                    yield itertools.chain((cell,), row_it)
                    break
    
def squeeze_nan(x):
    original_columns=x.index.tolist()

    squeezed=x.dropna()
    squeezed.index=[original_columns[n] for n in range(squeezed.count())]

    return squeezed.reindex(original_columns, fill_value=np.nan)

In [ ]:
# Correcting and labeling staff data. Naming certain columns not present in the other data type, job_titles.
# Extracting staff salary = max value
# Extracting staff R&D time spent = min value
# Ensuring staff R&D time is within range of a percentage

def staff_costs(ws_list):
    concatenated_costs=pd.concat(ws_list)
    concatenated_costs.reset_index(inplace=True)
    concatenated_costs.columns=['names', 'job_titles', 'costs', 'apportionment', 'r&d cost','filename', 'worksheet']
    concatenated_costs['job_titles']=concatenated_costs['job_titles'].astype(str)
    concatenated_costs['job_titles']=concatenated_costs['job_titles'].str.findall(job_titles)
    concatenated_costs['job_titles']=concatenated_costs['job_titles'].apply(','.join)
    concatenated_costs['job_titles']=concatenated_costs['job_titles'].str.replace('\,', ' ')
    concatenated_costs=concatenated_costs[concatenated_costs['job_titles'] != '']
    concatenated_costs=concatenated_costs[pd.to_numeric(concatenated_costs['apportionment'], errors='coerce').notnull()]
    concatenated_costs=concatenated_costs[concatenated_costs['apportionment'].apply(lambda x: type(x) in [int, np.int64, float, np.float64])]
    concatenated_costs=concatenated_costs[concatenated_costs['costs'].apply(lambda x: type(x) in [int, np.int64, float, np.float64])]
    concatenated_costs['costs'] = concatenated_costs.apply(lambda x: pd.to_numeric(x, errors='coerce')).max(axis=1)
    concatenated_costs['pct'] = concatenated_costs.apply(lambda x: pd.to_numeric(x, errors='coerce')).min(axis=1)
    concatenated_costs=concatenated_costs[concatenated_costs['pct'].between(0.1, 100, inclusive=True)]
    concatenated_costs.dropna(axis=0, how='any', inplace=True)
    return concatenated_costs

In [ ]:
# Adding identifier data that can connect the cost data to client data.
#Identifier data extracting from looking up the file names in the original folder in the server, 
# extracting the Cif identifier from matching the filename from the original folder to the spreadsheet filename.
# If there are multiple cifs then tidy split will extract them and split to new rows.

def tidy_split(df, column, sep=',', keep=False):

    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    return new_df
          

def client_id(corrected_data):
    corrected=pd.DataFrame()
    filedir=pd.DataFrame()
    listtss=[]
    for folderName, subfolders, filenames in os.walk(
        r'T:\R&D\Reports'):

        for summary in filenames:
            if 'Costs Summary' in summary or 'Cost Summary' in summary:
                listtss.append([summary, folderName])

    filedir=pd.DataFrame(listtss, columns=['filename', 'foldername'])
    filedir['filename']=filedir['filename'].astype(str)
    corrected=corrected_data.reset_index().merge(filedir, on='filename').set_index('costs')
    corrected['foldername']=corrected['foldername'].astype(str)
    corrected['Cif']=corrected['foldername'].str.findall(cif_regex)
    corrected=tidy_split(corrected, 'Cif', sep=',')
    corrected['Cif']=corrected['Cif'].astype(str).str.replace('\[|\]|\'|\-|\s', '')
    corrected=corrected[corrected['Cif'] != '']
    return corrected


In [ ]:
# regex to get the yearly period of the expenditure from the worksheet of the cost summary.

def year(finished):
    yydd_regex=re.compile(r'\d{2}(?=\D\D)')
    yy_regex=re.compile(r'\d{2}$')
    finished.worksheet=finished.worksheet.astype(str)
    finished['yydd']=finished.worksheet.str.findall(yydd_regex)
    finished['ye']=finished.yydd.apply(','.join)
    finished['yy']=finished['ye'].str.findall(yy_regex)
    finished['ye']=finished.yy.apply(','.join)
    finished['ye']
    finished
    return finished

In [ ]:
# Processing specifically for non staff costs. This method ensures that all relevant columns are extracted
# staff costs can be extracted in four columns.
# The same treatment otherwise like the staff_costs() function

def non_staff_costs(wx_list):
    concatenated_costs=pd.concat(wx_list)
    concatenated_costs=concatenated_costs[[1, 2, 3, 4, 5, 'filename', 'Worksheet']]
    concatenated_costs.reset_index(inplace=True)
    concatenated_costs.columns=['name','costs', 'apportionment', 'r&d cost', 'qualifying', 'total', 'filename', 'worksheet']
    concatenated_costs['name']=concatenated_costs['name'].astype(str)
    concatenated_costs['name']=concatenated_costs['name'].str.findall(job_titles)
    concatenated_costs['name']=concatenated_costs['name'].apply(','.join)
    concatenated_costs['name']=concatenated_costs['name'].str.replace('\,', ' ')
    concatenated_costs=concatenated_costs[concatenated_costs['name'] != '']
    concatenated_costs=concatenated_costs[pd.to_numeric(concatenated_costs['apportionment'], errors='coerce').notnull()]
    concatenated_costs=concatenated_costs[concatenated_costs['apportionment'].apply(lambda x: type(x) in [int, np.int64, float, np.float64])]
    concatenated_costs=concatenated_costs[concatenated_costs['costs'].apply(lambda x: type(x) in [int, np.int64, float, np.float64])]
    concatenated_costs['costs'] = concatenated_costs.apply(lambda x: pd.to_numeric(x, errors='coerce')).max(axis=1)
    concatenated_costs['pct'] = concatenated_costs.apply(lambda x: pd.to_numeric(x, errors='coerce')).min(axis=1)
    concatenated_costs=concatenated_costs[concatenated_costs['pct'].between(0.1, 100, inclusive=True)]
    concatenated_costs=concatenated_costs[concatenated_costs['costs'] > 100]
    return concatenated_costs

In [3]:
# extraction of staff costs. Looping over each workbook, worksheet. 
# locating the data and relevant worksheets, changing the data to make the data we want extractable.
# Extracting that data from its positions in the worksheets and adding the source of the data.
# Only the relevant columns are selected and added to a list for further processing.
# The functions above are applied to the data, any duplicates are removed, and if there are duplicates only the
# last is selected, assuming that if there are different versions then the latest version is chosen.
#Only the cost data relevant to the prediction model is selected, cost and apportionment.
 
def staff_extract_generator(raw_d):
    removed_nan=pd.DataFrame()
    text_extract=pd.DataFrame()
    try: 
        raw_d.dropna(axis=0, thresh=2, inplace=True)
        removed_nan=raw_d.apply(squeeze_nan, axis=1)
        removed_nan.set_index(removed_nan.columns[0], inplace=True)
        removed_nan.index.str.strip()
        staff_extract= removed_nan.loc[:"Grand Total of Claimable Costs"].copy()
        index_costs=staff_extract.index.to_series()
        index_costs=index_costs.astype(str)
        index_costs=index_costs.str.lower()
        index_costs=index_costs.apply(lambda x: 's total costs' if 'staff costs relating to r&d' in x else x)
        index_costs=index_costs.apply(lambda x: 'staff costs' if 'staff' in x else x)
        staff_extract.index=index_costs
        staff_extract= staff_extract.loc["staff costs" : "s total costs"].copy()
        staff_extract['filename']=os.path.basename(summary)
        staff_extract['Worksheet']=ws
        text_extract=staff_extract[[1, 2, 3, 4, 'filename', 'Worksheet']]

    except Exception as e: print(e)

    return text_extract

def main_staff_costs():
    pass

if __name__ == '__main__':
    main_staff_costs()
    for summary in all_files:
        try: wb=load_workbook(summary, data_only=True)
        except Exception as e: print(e)
        for sheet in wb.worksheets:
            ws=sheet
            raw_d=DataFrame(get_data(ws))  
            text_extract=staff_extract_generator(raw_d)
            ws_list.append(text_extract)
    corrected_data=staff_costs(ws_list)
    finished=client_id(corrected_data)
    done=year(finished)
    done.drop_duplicates(subset=['names', 'worksheet'], keep='last', inplace=True)
    done.reset_index(inplace=True)
    app_db=done[['pct', 'Cif', 'ye']]
    cap_analysis=done[['costs', 'pct', 'Cif', 'ye']]
    cap_analysis.to_csv('staff_costs.csv', sep=',', encoding='utf-8')
    

'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'


C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\reader\worksheet.py:310: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
Cannot set a frame with no defined index and a value that cannot be converted to a Series
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Cla

'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
index 0 is out of bounds for axis 0 with size 0
'Grand Total of Claimable Costs'
index 0 is out of bounds for axis 0 with size 0
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
Cannot set a frame with no defined index and a value that cannot be converted t

In [4]:
cap_analysis.head()

,costs,pct,Cif,ye
0,17170.80,60.0,2232,14
1,13618.13,37.5,2232,14
2,14876.26,20.0,2232,14
3,14196.00,20.0,2232,14
4,24388.60,65.0,2232,14


In [5]:
# The same as the staff costs above with subcontraction labels.
# The columns are selected later so we can account for additional columns in the data.
# As it is very possible that the same subcontractor could work for multiple parties,
# I added cif to ensure we do not de duplicate across multiple clients.
#Same selected columns and output as Staff Costs.

def sub_extract_generator(raw_d):
    removed_nan=pd.DataFrame()
    subcontract_extract=pd.DataFrame()
    try: 
        raw_d.dropna(axis=0, thresh=2, inplace=True)
        removed_nan=raw_d.apply(squeeze_nan, axis=1)
        removed_nan.set_index(removed_nan.columns[0], inplace=True)
        removed_nan.index.str.strip()
        subcontract_extract= removed_nan.loc[:"Grand Total of Claimable Costs"].copy()
        index_costs=subcontract_extract.index.to_series()
        index_costs=index_costs.astype(str)
        index_costs=index_costs.str.lower()
        index_costs=index_costs.apply(lambda x: 'xtotal sub' if 'total subcontract' in x else x)
        index_costs=index_costs.apply(lambda x: 'xsubcontractor' if 'subcontract' in x else x)
        subcontract_extract.index=index_costs
        subcontract_extract['Worksheet']=ws
        subcontract_extract=staff_extract.loc["xsubcontractor" : "xtotal sub"].copy()
        subcontract_extract['filename']=os.path.basename(summary)
        
    except Exception as e: print(e)

    return subcontract_extract


def main_sub_costs():
    pass

if __name__ == '__main__':
    main_sub_costs()
    for summary in all_files:
        try: wb=load_workbook(summary, data_only=True)
        except Exception as e: print(e)
        for sheet in wb.worksheets:
            ws=sheet
            raw_d=DataFrame(get_data(ws))  
            text_extract=sub_extract_generator(raw_d)
            ws_list.append(text_extract)
    corrected_data=non_staff_costs(ws_list)
    finished=client_id(corrected_data)
    done=year(finished)
    done.drop_duplicates(subset=['name', 'Cif', 'worksheet'], keep='last', inplace=True)
    done.reset_index(inplace=True)
    app_db=done[['pct', 'Cif', 'ye']]
    cap_analysis=done[['costs', 'pct', 'Cif', 'ye']]
    cap_analysis.to_csv('sub_costs.csv', sep=',', encoding='utf-8')

'xsubcontractor'
'xsubcontractor'
'Grand Total of Claimable Costs'
'xsubcontractor'
'xsubcontractor'
'Grand Total of Claimable Costs'


C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\reader\worksheet.py:310: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
"Cannot get right slice bound for non-unique label: 'xtotal sub'"
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
Cannot set a frame with no defined index and a value that cannot be converted to a Series
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'xsubcontractor'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
"Cannot get right slice bound for non-unique label: 'xtotal sub'"
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'G

"Cannot get right slice bound for non-unique label: 'xtotal sub'"
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
index 0 is out of bounds for axis 0 with size 0
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
"Cannot get right slice bound for non-unique label: 'xtotal sub'"
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
"Cannot get right slice bound for non-unique label: 'xtotal sub'"
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
"Cannot get right slice bound for non-unique label: 'xtotal sub'"
'Grand Total of Claimable Costs'
'Grand Total of Claimable Co

KeyError: 'names'

In [7]:
cap_analysis.head()

,costs,pct,Cif,ye
0,4978.31,100.0,3180,15
1,7800.00,90.0,4136,16
2,75343.74,70.0,504,13
3,42728.89,70.0,504,13
4,42728.89,60.0,504,13


In [8]:
# The same process as for the subcontractors but with different indexes.
def consum_extract_generator(raw_d):
    removed_nan=pd.DataFrame()
    consum_extract=pd.DataFrame()
    try: 
        raw_d.dropna(axis=0, thresh=2, inplace=True)
        removed_nan=raw_d.apply(squeeze_nan, axis=1)
        removed_nan.set_index(removed_nan.columns[0], inplace=True)
        removed_nan.index.str.strip()
        consum_extract= removed_nan.loc[:"Grand Total of Claimable Costs"].copy()
        index_costs=staff_extract.index.to_series()
        index_costs=index_costs.astype(str)
        index_costs=index_costs.str.lower()
        index_costs=index_costs.apply(lambda x: 'total consumes' if 'consumables costs relating to r&d' in x else x)
        index_costs=index_costs.apply(lambda x: 'consumes' if 'consumables' in x else x)
        consum_extract.index=index_costs
        consum_extract['Worksheet']=ws
        consum_extract= consum_extract.loc["consumes" : "total consumes"].copy()
        consum_extract['filename']=os.path.basename(summary)
        
    except Exception as e: print(e)

    return consum_extract

def main_consum_costs():
    pass

if __name__ == '__main__':
    main_consum_costs()
    for summary in all_files:
        try: wb=load_workbook(summary, data_only=True)
        except Exception as e: print(e)
        for sheet in wb.worksheets:
            ws=sheet
            raw_d=DataFrame(get_data(ws))  
            text_extract=consum_extract_generator(raw_d)
            ws_list.append(text_extract)
    corrected_data=non_staff_costs(ws_list)
    finished=client_id(corrected_data)
    done=year(finished)
    done.drop_duplicates(subset=['name', 'Cif', 'worksheet'], keep='last', inplace=True)
    done.reset_index(inplace=True)
    app_db=done[['pct', 'Cif', 'ye']]
    cap_analysis=done[['costs', 'pct', 'Cif', 'ye']]
    cap_analysis.to_csv('consum_costs.csv', sep=',', encoding='utf-8')

'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'


C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\reader\worksheet.py:310: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
Cannot set a frame with no defined index and a value that cannot be converted to a Series
'Grand Total of Claimable Costs'
'consumes'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'consumes'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Cost

index 0 is out of bounds for axis 0 with size 0
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Co

In [9]:
cap_analysis.head()

,costs,pct,Cif,ye
0,4978.31,100.0,3180,15
1,7800.00,90.0,4136,16
2,3127.00,100.0,4136,15
3,11157.00,100.0,4136,16
4,75343.74,70.0,504,13


In [12]:
# The same process as for subcontraction and consumable costs but with software labels.

def soft_extract_generator(raw_d):
    removed_nan=pd.DataFrame()
    consum_extract=pd.DataFrame()
    try: 
        raw_d.dropna(axis=0, thresh=2, inplace=True)
        removed_nan=raw_d.apply(squeeze_nan, axis=1)
        removed_nan.set_index(removed_nan.columns[0], inplace=True)
        removed_nan.index.str.strip()
        soft_extract=removed_nan.loc[:"Grand Total of Claimable Costs"].copy()
        index_costs=staff_extract.index.to_series()
        index_costs=index_costs.astype(str)
        index_costs=index_costs.str.lower()
        index_costs=index_costs.apply(lambda x: 'total software' if 'total software' in x else x)
        index_costs=index_costs.apply(lambda x: 'software costs' if 'software costs' in x else x)
        soft_extract.index=index_costs
        soft_extract['Worksheet']=ws
        soft_extract= soft_extract.loc["software costs" : "total software"].copy()
        soft_extract['filename']=os.path.basename(summary)
        
    except Exception as e: print(e)

    return soft_extract

def main_soft_costs():
    pass

if __name__ == '__main__':
    main_soft_costs()
    for summary in all_files:
        try: wb=load_workbook(summary, data_only=True)
        except Exception as e: print(e)
        for sheet in wb.worksheets:
            ws=sheet
            raw_d=DataFrame(get_data(ws))  
            text_extract=soft_extract_generator(raw_d)
            ws_list.append(text_extract)
    corrected_data=non_staff_costs(ws_list)
    finished=client_id(corrected_data)
    done=year(finished)
    done.drop_duplicates(subset=['name', 'Cif', 'worksheet'], keep='last', inplace=True)
    done.reset_index(inplace=True)
    app_db=done[['pct', 'Cif', 'ye']]
    cap_analysis=done[['costs', 'pct', 'Cif', 'ye']]
    cap_analysis.to_csv('soft_costs.csv', sep=',', encoding='utf-8')

'software costs'
'software costs'
'Grand Total of Claimable Costs'
'software costs'
'software costs'
'Grand Total of Claimable Costs'


C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\reader\worksheet.py:310: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'software costs'
'software costs'
Cannot set a frame with no defined index and a value that cannot be converted to a Series
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'software costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'software costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'software costs'


index 0 is out of bounds for axis 0 with size 0
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
index 0 is out of bounds for axis 0 with size 0
'Grand Total of Claimable Costs'
index 0 is out of bounds for axis 0 with size 0
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs'
'Grand Total of Claimable Costs

In [13]:
cap_analysis.head()

,costs,pct,Cif,ye
0,4978.31,100.0,3180,15
1,7800.00,90.0,4136,16
2,3127.00,100.0,4136,15
3,11157.00,100.0,4136,16
4,75343.74,70.0,504,13
